In [ ]:
from DeepFMKit.helpers import calculate_m_precision
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.cm import viridis
from DeepFMKit.plotting import default_rc
plt.rcParams.update(default_rc)

In [ ]:

def plot_precision_vs_ndata():
    """
    Plots the statistical uncertainty in 'm' as a function of the number of
    harmonics used in the fit.
    """
    # --- Analysis Parameters ---
    snr_db_fixed = 80.0
    m_range = np.linspace(0.1, 30.0, 500)
    ndata_range = np.arange(3, 31)
    
    # --- Plotting Setup ---
    fig, ax = plt.subplots(figsize=(6, 3), dpi=300)
    colors = viridis(np.linspace(0, 1, len(ndata_range)))

    print("Calculating uncertainty for different numbers of harmonics...")
    for i, ndata in enumerate(ndata_range):
        delta_m = calculate_m_precision(m_range, ndata, snr_db_fixed)
        ax.semilogy(m_range, delta_m, color=colors[i], label=f'ndata = {ndata}')

    # --- Aesthetics ---
    ax.set_xlabel('Modulation Depth (m)')
    ax.set_ylabel(r'Statistical Uncertainty ($\delta m$)')
    ax.set_title(f'DFMI Precision vs. Number of Harmonics (SNR = {snr_db_fixed} dB)')
    ax.grid(True, which='both', linestyle=':')
    # ax.set_ylim(1e-7, 1e-2) # Set a reasonable y-axis limit
    
    # Create a colorbar for the legend
    sm = plt.cm.ScalarMappable(cmap=viridis, norm=plt.Normalize(vmin=ndata_range.min(), vmax=ndata_range.max()))
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, ticks=ndata_range[::3]) # Show ticks every 3rd value
    cbar.set_label('Number of Harmonics (ndata)', rotation=270, labelpad=20)

    plt.tight_layout()
    return ax

ax = plot_precision_vs_ndata()
ax.set_ylim(0,1e-2)
plt.show()

In [ ]:
def plot_precision_vs_snr(ndata):
    """
    Plots the statistical uncertainty in 'm' as a function of the signal-to-noise
    ratio (SNR) of the measurement.
    """
    # --- Analysis Parameters ---
    ndata_fixed = ndata
    m_range = np.linspace(1.0, 30.0, 500)
    snr_db_range = np.linspace(20, 100, 11) # From 40 dB to 100 dB
    
    # --- Plotting Setup ---
    fig, ax = plt.subplots(figsize=(12, 7))
    colors = viridis(np.linspace(0, 1, len(snr_db_range)))

    print("Calculating uncertainty for different SNR values...")
    for i, snr_db in enumerate(snr_db_range):
        delta_m = calculate_m_precision(m_range, ndata_fixed, snr_db)
        ax.semilogy(m_range, delta_m, color=colors[i], label=f'SNR = {snr_db} dB')
        ax.axhline(y=np.sqrt(8)/10**(snr_db/20))

    # --- Aesthetics ---
    ax.set_xlabel('Modulation Depth (m)', fontsize=14)
    ax.set_ylabel(r'Statistical Uncertainty ($\delta m$)', fontsize=14)
    ax.set_title(f'DFMI Precision vs. Signal-to-Noise Ratio (ndata = {ndata_fixed})', fontsize=16)
    ax.grid(True, which='both', linestyle=':')
    # ax.set_ylim(1e-8, 1e-2)
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    plot_precision_vs_snr(100)